---

## ⚗️ LCEL 핵심 개념 이해하기

### 🔗 파이프라인(Pipeline)이란?

**LCEL의 핵심은 파이프라인 연산자 `|`** 입니다. 이는 **Unix 파이프라인**에서 영감을 받은 개념으로, 하나의 출력이 다음 단계의 입력으로 자연스럽게 전달됩니다.

#### 🏭 **공장 생산라인으로 이해하기**

```
📦 원재료 → 🔨 가공1 → ⚙️ 가공2 → 📋 검수 → ✅ 완제품
```

LCEL도 똑같은 방식으로 동작합니다:

```python
# 🔗 LCEL 체인 구성
chain = prompt_template | model | output_parser
#      ↑              ↑     ↑
#   1단계: 프롬프트 생성 → 2단계: AI 처리 → 3단계: 결과 파싱
```

### 🎯 기본 체인의 3단계 구조

#### 1️⃣ **PromptTemplate** - 📝 지시사항 준비
- **역할**: 사용자 입력을 AI가 이해할 수 있는 형태로 변환
- **입력**: 딕셔너리 형태의 변수들 (`{"topic": "인공지능"}`)
- **출력**: 완성된 프롬프트 문자열

#### 2️⃣ **Model** - 🤖 AI 처리 
- **역할**: 프롬프트를 받아 AI가 답변 생성
- **입력**: 포맷팅된 프롬프트 텍스트
- **출력**: AIMessage 객체 (내용 + 메타데이터)

#### 3️⃣ **OutputParser** - 🎁 결과 정리
- **역할**: AI 응답을 사용하기 쉬운 형태로 변환  
- **입력**: AIMessage 객체
- **출력**: 순수 텍스트 또는 구조화된 데이터

### 💡 파이프라인의 강력함

```python
# 각 단계를 개별적으로 실행하는 전통적 방식 ❌
formatted_prompt = prompt_template.format(country="대한민국")
ai_response = model.invoke(formatted_prompt)  
final_result = output_parser.parse(ai_response)

# LCEL로 한 번에 실행하는 방식 ✅
result = chain.invoke({"country": "대한민국"})
```

**한 줄로 전체 과정을 완료!** 이것이 LCEL의 매력입니다. 🚀

---

## 📝 PromptTemplate 완전 정복

**PromptTemplate**은 **동적 프롬프트**를 만들어주는 강력한 도구입니다. 마치 **편지 양식**에 이름만 바꿔서 여러 사람에게 보내는 것과 같습니다!

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

### 🔧 PromptTemplate의 구성 요소

**PromptTemplate**은 **템플릿 엔진**의 역할을 하며, 사용자 입력을 받아 **동적으로 프롬프트를 생성**합니다.

#### 📋 **핵심 구성 요소**

1. **🎯 template**: 실제 프롬프트 내용이 담긴 문자열 템플릿
   ```python
   template = "{country}의 수도는 어디인가요?"
   ```

2. **🔑 input_variables**: 템플릿에서 사용할 변수들의 이름 목록
   ```python
   # 중괄호 {} 안의 변수명들이 input_variables가 됨
   # 위 예시에서는 ["country"]가 자동으로 추출됨
   ```

#### 💡 **템플릿 변수 사용법**

- **중괄호 `{}`** 안에 변수명 작성
- **여러 변수** 사용 가능: `"{name}님, {city}의 날씨는 어떤가요?"`
- **변수명은 영문자로 시작**, 숫자와 언더스코어 사용 가능

#### 🚀 **from_template() 메서드의 편리함**

```python
# ✅ 간단한 방법 - from_template() 사용 (권장)
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# ❌ 복잡한 방법 - 직접 생성 (비권장)  
prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=["country"]
)
```

**from_template()의 장점:**
- **🔍 자동 변수 추출**: 중괄호 안의 변수를 자동으로 인식
- **✨ 간결한 코드**: 한 줄로 템플릿 생성 완료
- **🛡️ 오류 방지**: 변수명 오타나 누락 위험 최소화

실제로 PromptTemplate을 만들어보겠습니다! 💻

In [ ]:
# 스트리밍 출력을 위한 헬퍼 함수와 PromptTemplate 클래스 임포트
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate

### 🏗️ PromptTemplate 객체 생성하기

`from_template()` 메서드를 사용하면 간단하게 PromptTemplate 객체를 만들 수 있습니다.

In [ ]:
# 동적 프롬프트 템플릿 정의 - {country} 부분이 변수로 대체됨
template = "{country}의 수도는 어디인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
# 중괄호 안의 변수들이 자동으로 input_variables로 인식됨
prompt_template = PromptTemplate.from_template(template)

# 생성된 PromptTemplate 객체 확인
prompt_template

In [ ]:
# 템플릿에 구체적인 값을 대입하여 완성된 프롬프트 생성
prompt = prompt_template.format(country="대한민국")
print(f"생성된 프롬프트: {prompt}")

In [ ]:
# 다른 국가로 변경하여 프롬프트 생성 테스트
prompt = prompt_template.format(country="미국")
print(f"생성된 프롬프트: {prompt}")

In [ ]:
# ChatOpenAI 모델 임포트 및 초기화
from langchain_openai import ChatOpenAI

# OpenAI GPT 모델 객체 생성
model = ChatOpenAI(
    model="gpt-4.1",  # 사용할 모델 지정
    temperature=0.1,  # 창의성 조절 (낮을수록 일관된 답변)
)

In [ ]:
response = model.invoke(prompt)
print(response.content)

---

## 🔗 LCEL 체인 생성 - 파이프라인의 마법

### 🎯 LCEL(LangChain Expression Language) 심화 이해

![LCEL Pipeline](./images/lcel.png)

**LCEL의 핵심은 `|` (파이프) 연산자**입니다. 이를 통해 여러 구성 요소를 **체인처럼 연결**하여 하나의 통합된 워크플로우를 만들 수 있습니다.

### 🔄 파이프라인 연산자의 작동 원리

```python
chain = prompt_template | model | output_parser
```

#### 📊 데이터 흐름 과정

1. **📥 입력**: `{"topic": "인공지능"}` (딕셔너리)
2. **📝 1단계**: `prompt_template` → 완성된 프롬프트 텍스트
3. **🤖 2단계**: `model` → AIMessage 객체 (AI 응답)  
4. **🎁 3단계**: `output_parser` → 최종 텍스트 결과

### 🛠️ Unix 파이프라인과의 유사점

**Unix 명령어**와 개념이 매우 유사합니다:

```bash
# Unix 파이프라인 예시
cat file.txt | grep "keyword" | wc -l
```

```python
# LCEL 파이프라인 예시  
chain = prompt | model | parser
```

**공통점:**
- **⬅️ 왼쪽에서 오른쪽**으로 데이터 흐름
- **🔗 각 단계의 출력**이 다음 단계의 입력이 됨
- **🧩 모듈화**: 각 구성요소를 독립적으로 테스트하고 교체 가능

### 💡 LCEL이 혁신적인 이유

#### ✨ **자동 최적화**
- **병렬 처리**: 가능한 부분은 동시에 실행
- **메모리 효율성**: 중간 결과의 스마트한 관리
- **스트리밍**: 실시간 결과 출력 지원

#### 🔧 **개발자 친화적**
- **직관적 문법**: 데이터 흐름을 한눈에 파악
- **디버깅 용이**: 각 단계별 결과 추적 가능
- **재사용성**: 구성요소를 다른 체인에서도 활용

이제 실제로 간단한 체인을 만들어봅시다! 🚀

In [ ]:
# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요.")

# ChatOpenAI 모델 객체 생성
model = ChatOpenAI(model="gpt-4.1", temperature=0.1)

# 기본 체인 구성 (출력 파서 없이) - 프롬프트와 모델만 연결
chain = prompt | model

### ⚡ invoke() 메서드 - 체인 실행하기

**invoke()** 는 LCEL 체인을 실행하는 가장 기본적인 메서드입니다.

#### 📋 **사용법**
- **입력 형태**: Python 딕셔너리 `{"변수명": "값"}`
- **실행 방식**: 동기식 (결과가 나올 때까지 대기)
- **반환값**: 체인의 최종 출력 (출력 파서에 따라 달라짐)

In [ ]:
# 체인 실행을 위한 입력 딕셔너리 정의
# 키는 템플릿의 변수명과 일치해야 함
input = {"topic": "인공지능 모델의 학습 원리"}

In [ ]:
# LCEL 체인 실행: 프롬프트 생성 → 모델 처리 → AIMessage 반환
# invoke() 메서드로 전체 파이프라인을 한 번에 실행
result = chain.invoke(input)

# 결과 출력 (AIMessage 객체 형태로 반환됨)
result

### 🌊 스트리밍 출력의 강력함

**스트리밍**은 AI가 응답을 생성하는 **실시간 과정을 관찰**할 수 있게 해주는 기능입니다. 마치 사람이 말하는 것처럼 단어 하나씩 나타납니다!

In [ ]:
# 스트리밍 방식으로 체인 실행 - 실시간으로 응답 생성 과정 확인
answer = chain.stream(input)

# langchain_teddynote의 헬퍼 함수로 스트리밍 출력을 깔끔하게 표시
stream_response(answer)

---

## 🎁 OutputParser - 결과를 원하는 형태로

**OutputParser**는 AI의 복잡한 응답을 **사용하기 쉬운 형태로 변환**해주는 마지막 단계입니다.

In [ ]:
# 문자열 출력 파서 임포트
from langchain_core.output_parsers import StrOutputParser

# StrOutputParser 객체 생성 - AIMessage에서 순수 텍스트만 추출
output_parser = StrOutputParser()

### 🔗 완전한 체인 구성하기

이제 **3단계 파이프라인**을 완성해봅시다: **PromptTemplate → Model → OutputParser**

In [ ]:
# 완전한 LCEL 체인 구성: 프롬프트 → 모델 → 출력 파서
# 이제 결과가 AIMessage가 아닌 순수 문자열로 반환됨
chain = prompt | model | output_parser

In [ ]:
# 완성된 체인으로 invoke 실행 - 이제 순수 문자열이 반환됨
input = {"topic": "인공지능 모델의 학습 원리"}
result = chain.invoke(input)

# 결과 출력 (이제 문자열 형태로 깔끔하게 출력됨)
print("=== 완성된 체인 결과 ===")
print(result)

In [ ]:
# 완성된 체인으로 스트리밍 실행
answer = chain.stream(input)

print("=== 스트리밍 출력 ===")
# 실시간으로 문자열이 생성되는 과정을 관찰
stream_response(answer)

---

## 🎯 실전 프로젝트: 영어 회화 튜터 AI 만들기

이제 배운 내용을 활용해서 **실용적인 영어 학습 도우미**를 만들어봅시다! 

### 💡 프로젝트 개요

- **목표**: 상황별 영어 회화 생성 + 한글 번역 제공
- **특징**: 체계적인 포맷으로 학습 효과 극대화
- **활용**: 다양한 상황에 맞는 영어 표현 학습

In [ ]:
# 전문적인 영어 회화 튜터 프롬프트 템플릿 설계
template = """You are an experienced English conversation teacher with 10 years of expertise.
Create practical English conversations for the given situation with Korean translations.
Please follow the FORMAT exactly as shown below.

#SITUATION:
{question}

#FORMAT:
- English Conversation:
- Korean Translation:
- Useful Expressions:
- Cultural Notes (if applicable):
"""

# 개선된 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 객체 생성 (최신 모델 사용)
model = ChatOpenAI(model_name="gpt-4.1")

# 문자열 출력 파서 생성
output_parser = StrOutputParser()

In [ ]:
# 영어 회화 튜터 체인 구성
# 프롬프트 → 모델 → 출력 파서의 완전한 파이프라인
chain = prompt | model | output_parser

In [ ]:
# 첫 번째 상황: 식당에서 음식 주문하기
situation_1 = "저는 식당에 가서 음식을 주문하고 싶어요"

print("🍽️ === 식당 주문 상황 ===")
print(chain.invoke({"question": situation_1}))

In [ ]:
# 두 번째 상황: 스트리밍으로 실시간 학습 경험
situation_2 = "미국에서 피자 주문"

print("🍕 === 미국 피자 주문 상황 (스트리밍) ===")
# 스트리밍으로 영어 회화가 실시간으로 생성되는 과정 관찰
answer = chain.stream({"question": situation_2})
stream_response(answer)

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)